In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [32]:
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'Cookie': '_gcl_au=1.1.1275729757.1719396998; _ga_K8QWRE7TPK=GS1.1.1719917399.1.1.1719917424.0.0.0; gr_1_deviceId=88a2e024-80d5-4c87-860b-9b7f54297e6d; gr_1_lat=12.9129842; gr_1_lon=77.6421466; gr_1_locality=3; gr_1_landmark=undefined; city=; _gid=GA1.2.1794582858.1720533144; __cfruid=4d48598203d79fbca59db3781001827176ee158f-1720590758; _cfuvid=wPaJCrZt_4JgEd.BCM_KLmauATtW9Z_mFcSU7xkDZqA-1720590758283-0.0.1.1-604800000; _ga=GA1.1.315477064.1719396999; _ga_DDJ0134H6Z=GS1.2.1720590760.9.1.1720596341.60.0.0; _ga_JSMJG966C7=GS1.1.1720611757.10.0.1720611757.60.0.0; __cf_bm=WEiBHg_acE7OScMtFlARszXowfjxzhW3yghI4GnlMLs-1720613558-1.0.1.1-JnHaiw2wWQSFIDrnZthnvajHxEjx6olpg2GOIt7UWqclyse6oaYBTO9kEMpMliPftNC75OCOjbx8LXP.q1nknQ'}

In [33]:
def extract_product_info(page_content):
    soup = BeautifulSoup(page_content, 'html.parser')
    product = soup.find_all('div', class_='ProductAttribute__ProductAttributes-sc-dyoysr-2 hsIFeK')
    
    # Title of the item
    title_element = soup.find('span', class_='ProductInfoCard__BreadcrumbProductName-sc-113r60q-9 kkUsFb')
    title_value = title_element.text.strip() if title_element else None
    
    # Initialize empty lists to store the extracted data
    details = []
    descriptions = []

    for i in product:
        # Extracting the text content of the detail element
        detail_element = i.find('p', class_='ProductAttribute__ProductAttributesName-sc-dyoysr-5 heSLbJ')
        detail_value = detail_element.text.strip() if detail_element else None
        details.append(detail_value)

        # Extracting the text content of the description element
        description_element = i.find('div', class_='ProductAttribute__ProductAttributesDescription-sc-dyoysr-6 jJVAqC')
        description_value = description_element.text.strip() if description_element else None
        descriptions.append(description_value)

    # Create a DataFrame from the lists
    return pd.DataFrame({
        'Item': [title_value] * len(details),
        'Detail': details,
        'Description': descriptions
    })


In [34]:
all_pages_data = []

for code in range(1, 10000):  # Adjust the range to scrape the desired number of pages
    url = f'https://blinkit.com/prn/Harshal-Mehta/prid/{code}'
    response = requests.get(url, headers=header)
    if response.status_code == 200:
        page_data = extract_product_info(response.content)
        all_pages_data.append(page_data)
    else:
        continue

In [35]:
# Combine all page data into a single DataFrame
final_df = pd.concat(all_pages_data, ignore_index=True)

# Print DataFrame
final_df

,Item,Detail,Description
0,Nutrela Soya Mini Chunks,Soya Mini Chunks,Type
1,Nutrela Soya Mini Chunks,Key Features,Carefully picked for the highest qualityLow ch...
2,Nutrela Soya Mini Chunks,Ingredients,Defatted Soya
3,Nutrela Soya Mini Chunks,Unit,200 g
4,Nutrela Soya Mini Chunks,FSSAI License,10012031000033
5,Nutrela Soya Mini Chunks,Shelf Life,12 months
6,Nutrela Soya Mini Chunks,Manufacturer Details,RUCHI SOYA INDUSTRIES LTD. Talawali Chanda AB ...
7,Nutrela Soya Mini Chunks,Country of Origin,India
8,Nutrela Soya Mini Chunks,Customer Care Details,Email: [email protected]
9,Nutrela Soya Mini Chunks,Return Policy,"This Item is non-returnable. For a damaged, de..."


In [11]:
final_df.to_csv('Blinkit_20k.csv')

In [10]:
final_df['Item'].nunique()

2687